In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply, LSTM
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']
from helper import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
WINDOW_SIZE = 186
INPUT_FEAT = 1
OUTPUT_CLASS = 3
BS = 256
EP = 100
Pat = 6

# Mon_Term = 'val_AUC'
# Mode = 'max'

Mon_Term = 'val_loss'
Mode = 'min'

Model_Name = 'GAP_'
def load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
    return Parallel_GAP(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)

# results = 'LSTM_'
# def load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS):
#     return parallel_NN_LSTM(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)

results = 'Class_Results_' + Model_Name + Mon_Term
if not os.path.exists(str(results)):
    os.makedirs(str(results))

In [3]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [12]:
X = reshape(np.load('Data/TrainVal/X.npy'))
y = np.load('Data/TrainVal/y.npy')
X_val = reshape(np.load('Data/TrainVal/X_val.npy'))
y_val = np.load('Data/TrainVal/y_val.npy')

print (X.shape, X_val.shape, y.shape, y_val.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_val, return_counts=True))


y = to_categorical(np.load('Data/TrainVal/y.npy'))
y_val = to_categorical(np.load('Data/TrainVal/y_val.npy'))

X_test = np.load('Data/Original/test_data.npy')
y_test = X_test[:,-1]
X_test = reshape(X_test[:,:-1])

print (X_test.shape, y_test.shape)
print (np.unique(y_test, return_counts=True))

y_test = to_categorical(y_test)

(163820, 186, 1) (40955, 186, 1) (163820,) (40955,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([31181,  2358,  7416]))
(203187, 186, 1) (203187,)
(array([0., 1., 2.]), array([155008,  12564,  35615]))


In [6]:
# title = 'Original'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

In [7]:
X = reshape(np.load('Data/SMOTETOMEK/X.npy'))
y = np.load('Data/SMOTETOMEK/y.npy')
print (np.unique(y, return_counts=True))
y = to_categorical(y)
print (X.shape, y.shape)

# title = 'SMOTETOMEK'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

(array([0., 1., 2.]), array([125551, 125559, 125553]))
(376663, 186, 1) (376663, 3)


In [8]:
X = reshape(np.load('Data/SMOTEENN/X.npy'))
y = np.load('Data/SMOTEENN/y.npy')
print (np.unique(y, return_counts=True))
y = to_categorical(y)
print (X.shape, y.shape)

# title = 'SMOTEENN'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

(array([0., 1., 2.]), array([122941, 125410, 125193]))
(373544, 186, 1) (373544, 3)


In [9]:
X = reshape(np.load('Data/SMOTE/X.npy'))
y = np.load('Data/SMOTE/y.npy')
print (np.unique(y, return_counts=True))
y = to_categorical(y)
print (X.shape, y.shape)

# title = 'SMOTE'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

(array([0., 1., 2.]), array([125561, 125561, 125561]))
(376683, 186, 1) (376683, 3)


In [10]:
X = reshape(np.load('Data/ADASYN/X.npy'))
y = np.load('Data/ADASYN/y.npy')
print (np.unique(y, return_counts=True))
y = to_categorical(y)
print (X.shape, y.shape)

# title = 'ADASYN'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

(array([0., 1., 2.]), array([125561, 125895, 125677]))
(377133, 186, 1) (377133, 3)


In [11]:
X = reshape(np.load('Data/SVMSMOTE/X.npy'))
y = np.load('Data/SVMSMOTE/y.npy')
print (np.unique(y, return_counts=True))
y = to_categorical(y)
print (X.shape, y.shape)

# title = 'SVMSMOTE'

# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))

# model = load_model_train(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# cb = [keras.callbacks.EarlyStopping(monitor=Mon_Term, patience=Pat, verbose=1, mode=Mode), 
#       ModelCheckpoint(filename+'/check.h5', save_best_only=True, monitor=Mon_Term, mode=Mode, verbose=1)]

# history = model.fit(X, y, validation_data=(X_val, y_val), batch_size=BS, epochs=EP, callbacks=cb, verbose=1)
# save_the_model(filename, model)
# write_history(filename, history.history)
# plt.figure(figsize=(5.5,3.2))
# prediction(X_test, y_test, filename, title)

(array([0., 1., 2.]), array([125561, 125561, 125561]))
(376683, 186, 1) (376683, 3)


In [6]:
# Original Temp for bad results

# def load_modell(filename):
#     # load json and create model
#     json_file = open(str(filename)+'/model.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     # load weights into new model
#     model.load_weights(str(filename)+"/model.h5")
#     model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
#                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                   metrics=[tf.keras.metrics.BinaryAccuracy(name='Accuracy'), 
#                            tf.keras.metrics.Recall(name='Recall'),  
#                            tf.keras.metrics.Precision(name='Precision'), 
#                            tf.keras.metrics.AUC(num_thresholds=200, summation_method="interpolation", 
#                                                 name="AUC", dtype=None, curve="ROC", thresholds=None, 
#                                                 multi_label=True, label_weights=None)])
#     return model

# title = 'Original'
# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))
    
# model = load_modell(filename)
# print ('Model Loaded')
# y_pred=model.predict(X_test,verbose=0)

# y_pred_arg = np.argmax(y_pred, axis=1)
# y_test_arg = np.argmax(y_test, axis=1)

# # y_pred=model.predict(X_test,verbose=1)
# cm = ConfusionMatrix(actual_vector=y_test_arg, predict_vector=y_pred_arg)
# totalt = cm.__dict__

# TP = totalt['TP']
# FP = totalt['FP']
# TN = totalt['TN']
# FN = totalt['FN']

# PPV = totalt['PPV']
# ACC = totalt['ACC']
# SEN = totalt['TPR']
# SPE = totalt['TNR']
# F1S = totalt['F1']
# AUC = totalt['AUC']

# f = open(str(filename)+'/Results.csv', "w")
# f.write('TP,FP,TN,FN,Precision,Accuracy,Sensitivity,Specificity,F1Score,AUC\n')
# for i in range(3):
#     f.write(str(TP[i])+','+str(FP[i])+','+str(TN[i])+','+str(FN[i])+','+str(PPV[i])+','+
#             str(ACC[i])+','+str(SEN[i])+','+str(SPE[i])+','+str(F1S[i])+','+str(AUC[i])+'\n')
# f.close()

# cnf_matrix = confusion_matrix(y_test_arg, y_pred_arg)
# classes = ['Normal','SVEB','VEB']
# plt.figure(figsize=(8,5))
# plot_confusion_matrix(cnf_matrix, classes, filename, title)

In [6]:
# # SMOTE Temp for bad results

# def load_modell(filename):
#     # load json and create model
#     json_file = open(str(filename)+'/model.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     # load weights into new model
#     model.load_weights(str(filename)+"/model.h5")
#     model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
#                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#                   metrics=[tf.keras.metrics.BinaryAccuracy(name='Accuracy'), 
#                            tf.keras.metrics.Recall(name='Recall'),  
#                            tf.keras.metrics.Precision(name='Precision'), 
#                            tf.keras.metrics.AUC(num_thresholds=200, summation_method="interpolation", 
#                                                 name="AUC", dtype=None, curve="ROC", thresholds=None, 
#                                                 multi_label=True, label_weights=None)])
#     return model

# title = 'SMOTE'
# filename = os.path.join(results,title)
# if not os.path.exists(str(filename)):
#     os.makedirs(str(filename))
    
# model = load_modell(filename)
# print ('Model Loaded')
# y_pred=model.predict(X_test,verbose=0)

# y_pred_arg = np.argmax(y_pred, axis=1)
# y_test_arg = np.argmax(y_test, axis=1)

# # y_pred=model.predict(X_test,verbose=1)
# cm = ConfusionMatrix(actual_vector=y_test_arg, predict_vector=y_pred_arg)
# totalt = cm.__dict__

# TP = totalt['TP']
# FP = totalt['FP']
# TN = totalt['TN']
# FN = totalt['FN']

# PPV = totalt['PPV']
# ACC = totalt['ACC']
# SEN = totalt['TPR']
# SPE = totalt['TNR']
# F1S = totalt['F1']
# AUC = totalt['AUC']

# f = open(str(filename)+'/Results.csv', "w")
# f.write('TP,FP,TN,FN,Precision,Accuracy,Sensitivity,Specificity,F1Score,AUC\n')
# for i in range(3):
#     f.write(str(TP[i])+','+str(FP[i])+','+str(TN[i])+','+str(FN[i])+','+str(PPV[i])+','+
#             str(ACC[i])+','+str(SEN[i])+','+str(SPE[i])+','+str(F1S[i])+','+str(AUC[i])+'\n')
# f.close()

# cnf_matrix = confusion_matrix(y_test_arg, y_pred_arg)
# classes = ['Normal','SVEB','VEB']
# plt.figure(figsize=(8,5))
# plot_confusion_matrix(cnf_matrix, classes, filename, title)

Model Loaded
